# Stage I GAN

# Importing Libraries

In [0]:
import os
import pickle
import random
import time

import PIL
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from keras import Input, Model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers import Dense, LeakyReLU, BatchNormalization, ReLU, Reshape, UpSampling2D, Conv2D, Activation, \
    concatenate, Flatten, Lambda, Concatenate
from keras.optimizers import Adam
from matplotlib import pyplot as plt

Using TensorFlow backend.


# Loading of Dataset

In [0]:
def load_class_ids(class_info_file_path):
    """
    Load class ids from class_info.pickle file
    """
    with open(class_info_file_path, 'rb') as f:
        class_ids = pickle.load(f, encoding='latin1')
        return class_ids

In [0]:
def load_embeddings(embeddings_file_path):
    """
    Load embeddings
    """
    with open(embeddings_file_path, 'rb') as f:
        embeddings = pickle.load(f, encoding='latin1')
        embeddings = np.array(embeddings)
        print('embeddings: ', embeddings.shape)
        print(embeddings[0])
    return embeddings

In [0]:
def load_filenames(filenames_file_path):
    """
    Load filenames.pickle file and return a list of all file names
    """
    with open(filenames_file_path, 'rb') as f:
        filenames = pickle.load(f, encoding='latin1')
    return filenames

In [0]:
'''
def load_bounding_boxes(dataset_dir):
    """
    Load bounding boxes and return a dictionary of file names and corresponding bounding boxes
    """
    # Paths
    bounding_boxes_path = os.path.join(dataset_dir, 'bounding_boxes.txt')
    file_paths_path = os.path.join(dataset_dir, 'images.txt')

    # Read bounding_boxes.txt and images.txt file
    df_bounding_boxes = pd.read_csv(bounding_boxes_path,
                                    delim_whitespace=True, header=None).astype(int)
    df_file_names = pd.read_csv(file_paths_path, delim_whitespace=True, header=None)

    # Create a list of file names
    file_names = df_file_names[1].tolist()

    # Create a dictionary of file_names and bounding boxes
    filename_boundingbox_dict = {img_file[:-4]: [] for img_file in file_names[:2]}

    # Assign a bounding box to the corresponding image
    for i in range(0, len(file_names)):
        # Get the bounding box
        bounding_box = df_bounding_boxes.iloc[i][1:].tolist()
        key = file_names[i][:-4]
        filename_boundingbox_dict[key] = bounding_box

    return filename_boundingbox_dict
'''

'\ndef load_bounding_boxes(dataset_dir):\n    """\n    Load bounding boxes and return a dictionary of file names and corresponding bounding boxes\n    """\n    # Paths\n    bounding_boxes_path = os.path.join(dataset_dir, \'bounding_boxes.txt\')\n    file_paths_path = os.path.join(dataset_dir, \'images.txt\')\n\n    # Read bounding_boxes.txt and images.txt file\n    df_bounding_boxes = pd.read_csv(bounding_boxes_path,\n                                    delim_whitespace=True, header=None).astype(int)\n    df_file_names = pd.read_csv(file_paths_path, delim_whitespace=True, header=None)\n\n    # Create a list of file names\n    file_names = df_file_names[1].tolist()\n\n    # Create a dictionary of file_names and bounding boxes\n    filename_boundingbox_dict = {img_file[:-4]: [] for img_file in file_names[:2]}\n\n    # Assign a bounding box to the corresponding image\n    for i in range(0, len(file_names)):\n        # Get the bounding box\n        bounding_box = df_bounding_boxes.iloc[i][

In [0]:
def get_img(img_path, image_size):
    """
    Load and resize image
    """
    img = Image.open(img_path).convert('RGB')
    
    width, height = img.size
    #print("wh:", img.size)
    '''
    if bbox is not None:
        R = int(np.maximum(bbox[2], bbox[3]) * 0.75)
        center_x = int((2 * bbox[0] + bbox[2]) / 2)
        center_y = int((2 * bbox[1] + bbox[3]) / 2)
        y1 = np.maximum(0, center_y - R)
        y2 = np.minimum(height, center_y + R)
        x1 = np.maximum(0, center_x - R)
        x2 = np.minimum(width, center_x + R)
        img = img.crop([x1, y1, x2, y2])
        
    '''   
    img = img.resize(image_size, PIL.Image.BILINEAR)
    
    from matplotlib.pyplot import imshow
    
    #pil_im = Image.open('data/empire.jpg', 'r')
    #imshow(np.asarray(img))
    
    #print(np.array(img).shape)
    
    #img.show()
    
    return img

In [0]:
def load_dataset(filenames_file_path, class_info_file_path, cub_dataset_dir, embeddings_file_path, image_size):
    """
    Load dataset
    """
    filenames = load_filenames(filenames_file_path)
    class_ids = load_class_ids(class_info_file_path)
    #bounding_boxes = load_bounding_boxes(cub_dataset_dir)
    all_embeddings = load_embeddings(embeddings_file_path)

    X, y, embeddings = [], [], []

    print("Embeddings shape:", all_embeddings.shape)

    for index, filename in enumerate(filenames):
        #bounding_box = bounding_boxes[filename]

        try:
            # Load images
            img_name = '{}/{}.jpg'.format(cub_dataset_dir, filename[:-4]) 
            img = get_img(img_name, image_size)

            all_embeddings1 = all_embeddings[index, :, :]

            embedding_ix = random.randint(0, all_embeddings1.shape[0] - 1)
            embedding = all_embeddings1[embedding_ix, :]

            X.append(np.array(img))
            y.append(class_ids[index])
            embeddings.append(embedding)
        except Exception as e:
            print(e)

    X = np.array(X)
    y = np.array(y)
    embeddings = np.array(embeddings)
    return X, y, embeddings

# Model Creation

In [0]:
def generate_c(x):
    mean = x[:, :128]
    log_sigma = x[:, 128:]
    stddev = K.exp(log_sigma)
    epsilon = K.random_normal(shape=K.constant((mean.shape[1],), dtype='int32'))
    c = stddev * epsilon + mean
    return c

In [0]:
def build_ca_model():
    """
    Get conditioning augmentation model.
    Takes an embedding of shape (1024,) and returns a tensor of shape (256,)
    """
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    model = Model(inputs=[input_layer], outputs=[x])
    return model

In [0]:
def build_embedding_compressor_model():
    """
    Build embedding compressor model
    """
    input_layer = Input(shape=(1024,))
    x = Dense(128)(input_layer)
    x = ReLU()(x)

    model = Model(inputs=[input_layer], outputs=[x])
    return model

In [0]:
def build_stage1_generator():
    """
    Builds a generator model used in Stage-I
    """
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    mean_logsigma = LeakyReLU(alpha=0.2)(x)

    c = Lambda(generate_c)(mean_logsigma)

    input_layer2 = Input(shape=(100,))

    gen_input = Concatenate(axis=1)([c, input_layer2])

    x = Dense(128 * 8 * 4 * 4, use_bias=False)(gen_input)
    x = ReLU()(x)

    x = Reshape((4, 4, 128 * 8), input_shape=(128 * 8 * 4 * 4,))(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(512, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(3, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = Activation(activation='tanh')(x)

    stage1_gen = Model(inputs=[input_layer, input_layer2], outputs=[x, mean_logsigma])
    return stage1_gen


In [0]:
def build_stage1_discriminator():
    """
    Create a model which takes two inputs
    1. One from above network
    2. One from the embedding layer
    3. Concatenate along the axis dimension and feed it to the last module which produces final logits
    """
    input_layer = Input(shape=(64, 64, 3))

    x = Conv2D(64, (4, 4),
               padding='same', strides=2,
               input_shape=(64, 64, 3), use_bias=False)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(128, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(256, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(512, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    input_layer2 = Input(shape=(4, 4, 128))

    merged_input = concatenate([x, input_layer2])

    x2 = Conv2D(64 * 8, kernel_size=1,
                padding="same", strides=1)(merged_input)
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha=0.2)(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1)(x2)
    x2 = Activation('sigmoid')(x2)

    stage1_dis = Model(inputs=[input_layer, input_layer2], outputs=[x2])
    return stage1_dis

In [0]:
def build_adversarial_model(gen_model, dis_model):
    input_layer = Input(shape=(1024,))
    input_layer2 = Input(shape=(100,))
    input_layer3 = Input(shape=(4, 4, 128))

    x, mean_logsigma = gen_model([input_layer, input_layer2])

    dis_model.trainable = False
    valid = dis_model([x, input_layer3])

    model = Model(inputs=[input_layer, input_layer2, input_layer3], outputs=[valid, mean_logsigma])
    return model

# Defining Loss

In [0]:
def KL_loss(y_true, y_pred):
    mean = y_pred[:, :128]
    logsigma = y_pred[:, :128]
    loss = -logsigma + .5 * (-1 + K.exp(2. * logsigma) + K.square(mean))
    loss = K.mean(loss)
    return loss

In [0]:

def custom_generator_loss(y_true, y_pred):
    # Calculate binary cross entropy loss
    return K.binary_crossentropy(y_true, y_pred)


In [0]:
def save_rgb_img(img, path):
    """
    Save an rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()

In [0]:
def write_log(callback, name, loss, batch_no):
    """
    Write training summary to TensorBoard
    """
    summary = tf.Summary()
    summary_value = summary.value.add()
    summary_value.simple_value = loss
    summary_value.tag = name
    callback.writer.add_summary(summary, batch_no)
    callback.writer.flush()

# Downloading of dataset in colab

In [0]:
#!wget http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz

In [0]:
#!ls
#!rm -r faces_em.zip

In [0]:
#lfw download
#!gdown https://drive.google.com/uc?id=1zymw8pVkxZ8mtr8A9ODgNvhP4UO7nnwj
# lfw-created_2 (400 images of our data)
!gdown https://drive.google.com/uc?id=1NB1deCJ4XZPvqhTXiRD1AG6FgKHu-h1Y

Downloading...
From: https://drive.google.com/uc?id=1NB1deCJ4XZPvqhTXiRD1AG6FgKHu-h1Y
To: /content/lfw-created2.zip
3.63MB [00:00, 112MB/s]


In [0]:
#birds dowload
#!gdown https://drive.google.com/uc?id=1yUc4rgG783dIYJS8PIiWiNNhsmAx650E
# faces emb
#!gdown https://drive.google.com/uc?id=1-RepGyGQhvcNlDpWfVoDlnLDHYMfIAdP
# faces emb test train
#!gdown https://drive.google.com/uc?id=16_NAbnGXAEEHuMTv6rwSAUnj-JlDPSMp
# faces emb test train norm
#!gdown https://drive.google.com/uc?id=1a5lgRJc7az4LEDpNJIL7uWoXfB59siC1
#
# faces_em_2 (embeddings of our data of 372)
!gdown https://drive.google.com/uc?id=1QrHl3tv7J89S9CyKz4m6_R8F4UXoKHh7

Downloading...
From: https://drive.google.com/uc?id=1QrHl3tv7J89S9CyKz4m6_R8F4UXoKHh7
To: /content/faces_em_2.zip
15.7MB [00:00, 95.5MB/s]


In [0]:
#! echo '0' > epoch.txt

In [0]:
'''import tarfile
tar = tarfile.open("CUB_200_2011.tgz")
tar.extractall()
tar.close()'''

'import tarfile\ntar = tarfile.open("CUB_200_2011.tgz")\ntar.extractall()\ntar.close()'

In [0]:
'''tar2 = tarfile.open("birds.tar.gz")
tar2.extractall()
tar2.close()'''


from zipfile import ZipFile


with ZipFile("lfw-created2.zip", 'r') as zip2:
  zip2.extractall()
  zip2.close()

In [0]:

with ZipFile("faces_em_2.zip", 'r') as zip1:
  zip1.extractall()
  zip1.close()

In [0]:
#!ls

In [0]:
#cd CUB_200_2011

In [0]:
#!ls

In [0]:
#os.chdir("/content/")

In [0]:
#!ls

In [0]:
!ls
!mkdir logs
!mkdir results
!ls

epoch.txt  faces_em_2.zip  lfw-created2.zip  stage1_dis.h5
faces_em   lfw-created2    sample_data	     stage1_gen.h5
epoch.txt  faces_em_2.zip  lfw-created2.zip  results	  stage1_dis.h5
faces_em   lfw-created2    logs		     sample_data  stage1_gen.h5


In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive/')

# Main File

In [0]:
'''  data_dir = "/content/faces_em/"
  #data_dir = "/content/gdrive/My Drive//major/birds/"
  train_dir = data_dir + "/train"
  test_dir = data_dir + "/test"
  image_size = 64
  batch_size = 64
  z_dim = 100
  stage1_generator_lr = 0.0002
  stage1_discriminator_lr = 0.0002
  stage1_lr_decay_step = 600
  epochs = 300
  condition_dim = 128

  embeddings_file_path_train = train_dir + "/embed.pickle"
  embeddings_file_path_test = test_dir + "/embed.pickle"

  filenames_file_path_train = train_dir + "/filenames.pickle"
  filenames_file_path_test = test_dir + "/filenames.pickle"

  class_info_file_path_train = train_dir + "/class_info.pickle"
  class_info_file_path_test = test_dir + "/class_info.pickle"

  cub_dataset_dir = "/content/lfw"
  #cub_dataset_dir = "/content/gdrive/My Drive//major/CUB_200_2011"

  # Define optimizers
  dis_optimizer = Adam(lr=stage1_discriminator_lr, beta_1=0.5, beta_2=0.999)
  gen_optimizer = Adam(lr=stage1_generator_lr, beta_1=0.5, beta_2=0.999)

  """"
  Load datasets
  """
  X_train, y_train, embeddings_train = load_dataset(filenames_file_path=filenames_file_path_train,
                                                    class_info_file_path=class_info_file_path_train,
                                                    cub_dataset_dir=cub_dataset_dir,
                                                    embeddings_file_path=embeddings_file_path_train,
                                                    image_size=(64, 64))

  X_test, y_test, embeddings_test = load_dataset(filenames_file_path=filenames_file_path_test,
                                                 class_info_file_path=class_info_file_path_test,
                                                 cub_dataset_dir=cub_dataset_dir,
                                                 embeddings_file_path=embeddings_file_path_test,
                                                 image_size=(64, 64))'''

'  data_dir = "/content/faces_em/"\n  #data_dir = "/content/gdrive/My Drive//major/birds/"\n  train_dir = data_dir + "/train"\n  test_dir = data_dir + "/test"\n  image_size = 64\n  batch_size = 64\n  z_dim = 100\n  stage1_generator_lr = 0.0002\n  stage1_discriminator_lr = 0.0002\n  stage1_lr_decay_step = 600\n  epochs = 300\n  condition_dim = 128\n\n  embeddings_file_path_train = train_dir + "/embed.pickle"\n  embeddings_file_path_test = test_dir + "/embed.pickle"\n\n  filenames_file_path_train = train_dir + "/filenames.pickle"\n  filenames_file_path_test = test_dir + "/filenames.pickle"\n\n  class_info_file_path_train = train_dir + "/class_info.pickle"\n  class_info_file_path_test = test_dir + "/class_info.pickle"\n\n  cub_dataset_dir = "/content/lfw"\n  #cub_dataset_dir = "/content/gdrive/My Drive//major/CUB_200_2011"\n\n  # Define optimizers\n  dis_optimizer = Adam(lr=stage1_discriminator_lr, beta_1=0.5, beta_2=0.999)\n  gen_optimizer = Adam(lr=stage1_generator_lr, beta_1=0.5, beta_

In [0]:
# dowloading stage1_gen.h5 for epoch 448
#!gdown https://drive.google.com/uc?id=1zX3UCjMwaRG96nMuYwPJ3s5jaizX6XgM

In [0]:

if __name__ == '__main__':
    data_dir = "/content/faces_em"
    #data_dir = "/content/gdrive/My Drive//major/birds/"
    train_dir = data_dir + "/train"
    test_dir = data_dir + "/test"
    image_size = 64
    batch_size = 10
    z_dim = 100
    stage1_generator_lr = 0.0002
    stage1_discriminator_lr = 0.0002
    stage1_lr_decay_step = 600
    epochs = 1000
    condition_dim = 128
    '''

    embeddings_file_path_train = train_dir + "/embed.pickle"
    embeddings_file_path_test = test_dir + "/embed.pickle"

    filenames_file_path_train = train_dir + "/filenames.pickle"
    filenames_file_path_test = test_dir + "/filenames.pickle"

    class_info_file_path_train = train_dir + "/class_info.pickle"
    class_info_file_path_test = test_dir + "/class_info.pickle"
    '''
    
    embeddings_file_path_train = train_dir + "/embed_train.pickle"
    embeddings_file_path_test = test_dir + "/embed_test.pickle"

    filenames_file_path_train = train_dir + "/filenames_train.pickle"
    filenames_file_path_test = test_dir + "/filenames_test.pickle"

    class_info_file_path_train = train_dir + "/class_info.pickle"
    class_info_file_path_test = test_dir + "/class_info.pickle"
    

    cub_dataset_dir = "/content/lfw-created2"
    #cub_dataset_dir = "/content/gdrive/My Drive/major/CUB_200_2011"
    
    # Define optimizers
    dis_optimizer = Adam(lr=stage1_discriminator_lr, beta_1=0.5, beta_2=0.999)
    gen_optimizer = Adam(lr=stage1_generator_lr, beta_1=0.5, beta_2=0.999)

    """"
    Load datasets
    """
    X_train, y_train, embeddings_train = load_dataset(filenames_file_path=filenames_file_path_train,
                                                      class_info_file_path=class_info_file_path_train,
                                                      cub_dataset_dir=cub_dataset_dir,
                                                      embeddings_file_path=embeddings_file_path_train,
                                                      image_size=(64, 64))

    X_test, y_test, embeddings_test = load_dataset(filenames_file_path=filenames_file_path_test,
                                                   class_info_file_path=class_info_file_path_test,
                                                   cub_dataset_dir=cub_dataset_dir,
                                                   embeddings_file_path=embeddings_file_path_test,
                                                   image_size=(64, 64))

    """
    Build and compile networks
    """
    ca_model = build_ca_model()
    ca_model.compile(loss="binary_crossentropy", optimizer="adam")

    stage1_dis = build_stage1_discriminator()
    stage1_dis.compile(loss='binary_crossentropy', optimizer=dis_optimizer)

    stage1_gen = build_stage1_generator()
    stage1_gen.compile(loss="mse", optimizer=gen_optimizer)

    embedding_compressor_model = build_embedding_compressor_model()
    embedding_compressor_model.compile(loss="binary_crossentropy", optimizer="adam")

    adversarial_model = build_adversarial_model(gen_model=stage1_gen, dis_model=stage1_dis)
    adversarial_model.compile(loss=['binary_crossentropy', KL_loss], loss_weights=[1, 2.0],
                              optimizer=gen_optimizer, metrics=None)

    tensorboard = TensorBoard(log_dir="logs/".format(time.time()))
    tensorboard.set_model(stage1_gen)
    tensorboard.set_model(stage1_dis)
    tensorboard.set_model(ca_model)
    tensorboard.set_model(embedding_compressor_model)

    # Generate an array containing real and fake values
    # Apply label smoothing as well
    real_labels = np.ones((batch_size, 1), dtype=float) * 0.9
    fake_labels = np.zeros((batch_size, 1), dtype=float) * 0.1
    
        
    # Resuming the training
    file1 = open("epoch.txt",'r') #directory change
    epoch_start = 0
    epoch_start = int(file1.read())
    if(epoch_start!=0):
      stage1_gen.load_weights("stage1_gen.h5") #directory change
      stage1_dis.load_weights("stage1_dis.h5") #directory change

    for epoch in range(epoch_start, epochs):
        print("========================================")
        print("Epoch is:", epoch)
        print("Number of batches", int(X_train.shape[0] / batch_size))

        gen_losses = []
        dis_losses = []
        
        #from google.colab import files
        #files.download("results1.zip")

        # Load data and train model
        number_of_batches = int(X_train.shape[0] / batch_size)
        for index in range(number_of_batches):
            print("Batch:{}".format(index+1))
            
            """
            Train the discriminator network
            """
            # Sample a batch of data
            z_noise = np.random.normal(0, 1, size=(batch_size, z_dim))
            image_batch = X_train[index * batch_size:(index + 1) * batch_size]
            embedding_batch = embeddings_train[index * batch_size:(index + 1) * batch_size]
            image_batch = (image_batch - 127.5) / 127.5

            # Generate fake images
            fake_images, _ = stage1_gen.predict([embedding_batch, z_noise], verbose=2)

            # Generate compressed embeddings
            compressed_embedding = embedding_compressor_model.predict_on_batch(embedding_batch)
            compressed_embedding = np.reshape(compressed_embedding, (-1, 1, 1, condition_dim))
            compressed_embedding = np.tile(compressed_embedding, (1, 4, 4, 1))

            dis_loss_real = stage1_dis.train_on_batch([image_batch, compressed_embedding],
                                                      np.reshape(real_labels, (batch_size, 1)))
            dis_loss_fake = stage1_dis.train_on_batch([fake_images, compressed_embedding],
                                                      np.reshape(fake_labels, (batch_size, 1)))
            dis_loss_wrong = stage1_dis.train_on_batch([image_batch[:(batch_size - 1)], compressed_embedding[1:]],
                                                       np.reshape(fake_labels[1:], (batch_size-1, 1)))

            d_loss = 0.5 * np.add(dis_loss_real, 0.5 * np.add(dis_loss_wrong, dis_loss_fake))

            print("d_loss_real:{}".format(dis_loss_real))
            print("d_loss_fake:{}".format(dis_loss_fake))
            print("d_loss_wrong:{}".format(dis_loss_wrong))
            print("d_loss:{}".format(d_loss))

            """
            Train the generator network 
            """
            g_loss = adversarial_model.train_on_batch([embedding_batch, z_noise, compressed_embedding],
                                                      [K.ones((batch_size, 1)) * 0.9, K.ones((batch_size, 256)) * 0.9])
            print("g_loss:{}".format(g_loss))

            dis_losses.append(d_loss)
            gen_losses.append(g_loss)

        """
        Save losses to Tensorboard after each epoch
        """
        write_log(tensorboard, 'discriminator_loss', np.mean(dis_losses), epoch)
        write_log(tensorboard, 'generator_loss', np.mean(gen_losses[0]), epoch)
        
        # Generate and save images after every 2nd epoch
        if epoch % 2 == 0:
            # z_noise2 = np.random.uniform(-1, 1, size=(batch_size, z_dim))
            z_noise2 = np.random.normal(0, 1, size=(batch_size, z_dim))
            embedding_batch = embeddings_test[0:batch_size]
            fake_images, _ = stage1_gen.predict_on_batch([embedding_batch, z_noise2])
            
            # saving weights as a checkpoint
            stage1_gen.save_weights("stage1_gen.h5") #directory change
            stage1_dis.save_weights("stage1_dis.h5") #directory change
            file = open("epoch.txt",'w') #directory change
            file.write(str(epoch))

            # Save images
            for i, img in enumerate(fake_images[:10]):
                save_rgb_img(img, "results/gen_{}_{}.png".format(epoch, i))
            
            #import shutil
            #shutil.make_archive("results", 'zip', "results")
            #shutil.make_archive("logs", 'zip', "logs")
                
        if epoch % 4 == 0:
          
            import shutil
            shutil.make_archive("results", 'zip', "results")
            shutil.make_archive("logs", 'zip', "logs")
            
            from google.colab import files
            #files.download("results.zip")
            #files.download("logs.zip")
            #files.download("epoch.txt")
            #files.download("stage1_dis.h5")
            #files.download("stage1_gen.h5")
            

    # Save models
    stage1_gen.save_weights("stage1_gen.h5")
    stage1_dis.save_weights("stage1_dis.h5")

Instructions for updating:
Colocations handled automatically by placer.
embeddings:  (300, 1, 1024)
[[ 0.0158      0.00724899  0.03270026 ...  0.02498868  0.01431595
  -0.02049159]]
Embeddings shape: (300, 1, 1024)
embeddings:  (73, 1, 1024)
[[ 0.01336752  0.00812143  0.03953188 ...  0.02536569  0.01195818
  -0.02577927]]
Embeddings shape: (73, 1, 1024)
Epoch is: 956
Number of batches 30
Batch:1
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


d_loss_real:0.3300873637199402
d_loss_fake:8.801533840596676e-05
d_loss_wrong:1.8126382201444358e-05
d_loss:0.16507021729012195
g_loss:[0.38413087, 0.3468923, 0.018619288]
Batch:2
d_loss_real:0.3332447111606598
d_loss_fake:1.543337345123291
d_loss_wrong:1.6385116396122612e-05
d_loss:0.5524607747793198
g_loss:[2.9569101, 2.9209285, 0.017990883]
Batch:3
d_loss_real:0.3338935673236847
d_loss_fake:0.07728280872106552
d_loss_wrong:2.7761690944316797e-05
d_loss:0.18627442605793476
g_loss:[2.9228864, 2.8858345, 0.018525902]
Batch:4
d_loss_real:0.36444100737571716
d_loss_fake:0.024546418339014053
d_loss_wrong:4.763221568282461e-06
d_loss:0.18835829896852374
g_loss:[3.4966571, 3.4601178, 0.018269677]
Batch:5
d_loss_real:0.32768750190734863
d_loss_fake:0.007281053811311722
d_loss_wrong:5.103018793306546e-06
d_loss:0.16566529020201415
g_loss:[2.6404629, 2.6040273, 0.018217798]
Batch:6
d_loss_real:0.33474427461624146
d_loss_fake:0.22335195541381836
d_loss_wrong:7.954021384648513e-06
d_loss:0.22321

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Epoch is: 957
Number of batches 30
Batch:1
d_loss_real:0.34797152876853943
d_loss_fake:0.001617782167159021
d_loss_wrong:3.7796069136675214e-06
d_loss:0.1743911548401229
g_loss:[1.6585381, 1.619211, 0.019663591]
Batch:2
d_loss_real:0.3896210193634033
d_loss_fake:6.644965537816461e-07
d_loss_wrong:1.257943131349748e-05
d_loss:0.19481382066373953
g_loss:[1.0127105, 0.9732254, 0.019742524]
Batch:3
d_loss_real:0.33626610040664673
d_loss_fake:0.2714621424674988
d_loss_wrong:1.029243958328152e-05
d_loss:0.236001156270504
g_loss:[7.8480363, 7.808795, 0.019620772]
Batch:4
d_loss_real:0.5318079590797424
d_loss_fake:1.000000082740371e-07
d_loss_wrong:4.365718268672936e-05
d_loss:0.26591491883573326
g_loss:[14.545355, 14.506285, 0.019534847]
Batch:5
d_loss_real:0.3429796099662781
d_loss_fake:4.413188071339391e-05
d_loss_wrong:1.0076530998048838e-05
d_loss:0.17150335708629427
g_loss:[5.181232, 5.142288, 0.019471863]
Batch:6
d_loss_real:0.3406734764575958
d_loss_fake:0.002933929208666086
d_loss_wro

...

Epoch is: 997
Number of batches 30
Batch:1
d_loss_real:0.32523202896118164
d_loss_fake:1.000000082740371e-07
d_loss_wrong:1.1984261618636083e-05
d_loss:0.16261903554595847
g_loss:[14.533954, 14.506285, 0.013834685]
Batch:2
d_loss_real:0.3251839280128479
d_loss_fake:1.000000082740371e-07
d_loss_wrong:1.6287731341435574e-05
d_loss:0.16259606093944967
g_loss:[14.533895, 14.506285, 0.013805062]
Batch:3
d_loss_real:0.32522010803222656
d_loss_fake:1.000000082740371e-07
d_loss_wrong:5.0520451623015106e-05
d_loss:0.1626227091292094
g_loss:[14.533837, 14.506285, 0.013776387]
Batch:4
d_loss_real:0.32553571462631226
d_loss_fake:1.000000082740371e-07
d_loss_wrong:5.273918759485241e-06
d_loss:0.162769200792809
g_loss:[14.533779, 14.506285, 0.013747036]
Batch:5
d_loss_real:0.325346440076828
d_loss_fake:1.000000082740371e-07
d_loss_wrong:3.57481294486206e-05
d_loss:0.16268218207096652
g_loss:[14.533723, 14.506285, 0.0137188565]
Batch:6
d_loss_real:0.32524698972702026
d_loss_fake:1.000000082740371e-07

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


g_loss:[0.49731356, 0.47178382, 0.012764868]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch is: 999
Number of batches 30
Batch:1
d_loss_real:0.32806000113487244
d_loss_fake:1.0351779167194763e-07
d_loss_wrong:7.201467269624118e-06
d_loss:0.164031826813698
g_loss:[0.39468443, 0.36904234, 0.012821054]
Batch:2
d_loss_real:0.32731422781944275
d_loss_fake:0.002762025222182274
d_loss_wrong:1.4586901670554653e-05
d_loss:0.16435126692522317
g_loss:[0.6171748, 0.5914794, 0.012847694]
Batch:3
d_loss_real:0.3263114392757416
d_loss_fake:1.1855647244374268e-05
d_loss_wrong:2.22618746192893e-05
d_loss:0.1631642490183367
g_loss:[0.38865542, 0.36300474, 0.012825346]
Batch:4
d_loss_real:0.3258127272129059
d_loss_fake:0.0009320119279436767
d_loss_wrong:6.392305294866674e-06
d_loss:0.16314096466521733
g_loss:[0.3603209, 0.3347165, 0.012802203]
Batch:5
d_loss_real:0.3266346752643585
d_loss_fake:1.000000082740371e-07
d_loss_wrong:8.255118700617459e-06
d_loss:0.1633194264118174
g_loss:[10.049665, 10.024101, 0.01278189]
Batch:6
d_loss_real:0.32573071122169495
d_loss_fake:0.02178061194717884
d

In [0]:
import shutil
shutil.make_archive("resultssss", 'zip', "results")
from google.colab import files
files.download("results.zip")

In [0]:
# download results
!zip -r /content/results.zip /content/results

from google.colab import files
files.download("/content/results.zip")

  adding: content/results/ (stored 0%)
  adding: content/results/gen_984_3.png (deflated 11%)
  adding: content/results/gen_976_2.png (deflated 9%)
  adding: content/results/gen_968_3.png (deflated 11%)
  adding: content/results/gen_974_5.png (deflated 9%)
  adding: content/results/gen_970_8.png (deflated 12%)
  adding: content/results/gen_982_3.png (deflated 11%)
  adding: content/results/gen_996_9.png (deflated 9%)
  adding: content/results/gen_984_5.png (deflated 10%)
  adding: content/results/gen_974_1.png (deflated 12%)
  adding: content/results/gen_978_6.png (deflated 9%)
  adding: content/results/gen_966_0.png (deflated 10%)
  adding: content/results/gen_956_9.png (deflated 9%)
  adding: content/results/gen_990_9.png (deflated 8%)
  adding: content/results/gen_988_4.png (deflated 8%)
  adding: content/results/gen_966_5.png (deflated 10%)
  adding: content/results/gen_978_1.png (deflated 9%)
  adding: content/results/gen_980_2.png (deflated 8%)
  adding: content/results/gen_978_4

In [0]:
# download logs
!zip -r /content/logs.zip /content/logs

files.download("/content/logs.zip")

  adding: content/logs/ (stored 0%)
  adding: content/logs/events.out.tfevents.1558077652.ea9e2f41bc2a (deflated 91%)
  adding: content/logs/events.out.tfevents.1558077651.ea9e2f41bc2a (deflated 92%)


In [0]:
#download weights/checkpoints
files.download("/content/epoch.txt")
files.download("/content/stage1_dis.h5")
files.download("/content/stage1_gen.h5")

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

     |████████████████████████████████| 993kB 3.4MB/s 


In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

KeyboardInterrupt: ignored

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'stage1_dis.h5'})
uploaded.SetContentFile('stage1_dis.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'stage1_gen.h5'})
uploaded.SetContentFile('stage1_gen.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))